
# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK 
### Please note
follow instruction 
https://github.com/astral-sh/uv

uv init
uv sync



In [114]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [115]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

In [116]:
import asyncio

In [117]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as server:
    fetch_tools = await server.list_tools(None,None)

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

## Extra installation step - if you don't have "node" on your computer

The next MCP tool uses node (the Javascript Server), and it needs you to have the command 'npx' installed on your computer.

You may already have this, but if not, here are super clear instructions on exactly what to do, courtesy of our friend.  
And thank you to student avid_learner for pointing this out.

https://chatgpt.com/share/68103af2-e2dc-8012-b259-bc135a23273b


In [32]:
import asyncio
from agents.mcp import MCPServerStdio

# Use MCP to call the Playwright tool via npx
playwright_params = {
    "command": "npx",
    "args": ["@playwright/mcp@latest"]
}

async def run_playwright_mcp():
    try:
        print("⏳ Launching Playwright MCP server...")
        async with MCPServerStdio(
            params=playwright_params,
            client_session_timeout_seconds=300  # wait up to 5 minutes
        ) as server:
            print("✅ Connected to Playwright MCP server.")

            # List available tools
            tools = await server.list_tools(None, None)
            print("🧰 Available tools:", tools)

            # Run a simple browser navigation
            result = await server.run_tool(tools[0], {
                "action": "goto",
                "url": "https://example.com"
            })

            print("🌐 Navigation Result:")
            print(result)

    except Exception as e:
        print("❌ Error:", str(e))

In [64]:
await run_playwright_mcp()

⏳ Launching Playwright MCP server...
✅ Connected to Playwright MCP server.
🧰 Available tools: [Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None), Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=N

### And now repeat for 2 more!

In [118]:
playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=120) as server:
    playwright_tools = await server.list_tools(None, None)

playwright_tools


[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_messages', title=None, desc

In [120]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as server:
    file_tools = await server.list_tools(None, None)

file_tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_multiple_files', title=None, description="Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.", inputSchema={'type': 'object', 'properties': {'paths': {'type': 'ar

In [74]:
from agents.mcp import MCPServerStdio
import os
import asyncio

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
os.makedirs(sandbox_path, exist_ok=True)

files_params = {
    "command": "node",
    "args": ["/usr/lib/node_modules/@modelcontextprotocol/server-filesystem/dist/index.js", sandbox_path]
}

async def run_filesystem_mcp():
    try:
        async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as server:
            file_tools = await server.list_tools(None, None)
            print("✅ Tools:", file_tools)
    except Exception as e:
        print("❌ MCP Server error:", e)



In [76]:
files_params

{'command': 'node',
 'args': ['/usr/lib/node_modules/@modelcontextprotocol/server-filesystem/dist/index.js',
  '/home/pradeepdhote/projects/MCP/MCP/sandbox']}

In [75]:
await run_filesystem_mcp()

✅ Tools: [Tool(name='read_file', title=None, description='Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None), Tool(name='read_multiple_files', title=None, description="Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.", inputSchema={'type': 'object', 'properties': {'paths': {'ty

### And now.. bring on the Agent with Tools!

In [ ]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-nano",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for apple Pie, then summarize it in markdown to applepie.md")
            print(result.final_output)

Error getting response: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-dsNDS9gDB4bWFMCiyfCHsre0 on tokens per min (TPM): Limit 10000, Used 8400, Requested 2531. Please try again in 5.586s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. (request_id: req_e1514461914e44710203202b136b5921)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-dsNDS9gDB4bWFMCiyfCHsre0 on tokens per min (TPM): Limit 10000, Used 8400, Requested 2531. Please try again in 5.586s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
# instructions = """
# You browse the internet to accomplish your instructions.
# You are highly capable at browsing the internet independently to accomplish your task, 
# including accepting all cookies and clicking 'not now' as
# appropriate to get to the content you need. If one website isn't fruitful, try another. 
# Be persistent until you have solved your assignment,
# trying different options and sites as needed.
# """


# async with MCPServerStdio(params=files_params, client_session_timeout_seconds=300) as mcp_server_files:
#     print("✅ Connected to filesystem MCP server.")
#     async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=300) as mcp_server_browser:
#         print("✅ Connected to Playwright MCP server.")
        
#         agent = Agent(
#             name="investigator", 
#             instructions=instructions, 
#             model="gpt-3.5-turbo-0125",
#             mcp_servers=[mcp_server_files, mcp_server_browser]
#         )
#         with trace("investigate"):
#             try:
#                 result = await Runner.run(agent, "Find a great recipe for apple Pie, then summarize it in markdown to applePie.md")
#                 print(result.final_output)
#                 print("✅ Investigation complete.")
#             except Exception as e:
#                 print("❌ Error during investigation:", e)
            
               

✅ Connected to filesystem MCP server.
✅ Connected to Playwright MCP server.


### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



